In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

class SimpleModel1D(nn.Module):
    def __init__(self, num_classes):
        super(SimpleModel1D, self).__init__()

        self.features = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),
            nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2)
        )

        self.adaptive_pool = nn.AdaptiveAvgPool1d(1)

        self.classifier = nn.Sequential(
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.adaptive_pool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Load the CSV files
train_df = pd.read_csv('./datasets/mitbih_dataset/mitbih_train.csv', header=None)
test_df = pd.read_csv('./datasets/mitbih_dataset/mitbih_test.csv', header=None)

# Map of original class labels
classes = {0: 'N', 1: 'S', 2: 'V', 3: 'F', 4: 'Q'}

# Add a new column for 'normal' vs 'not normal'
train_df['binary_label'] = train_df[187].apply(lambda x: 0 if x == 0 else 1)
test_df['binary_label'] = test_df[187].apply(lambda x: 0 if x == 0 else 1)

# Prepare the data
X_train = train_df.iloc[:, :187].values
y_train = train_df.iloc[:, 187].values
y_train_binary = train_df['binary_label'].values

X_test = test_df.iloc[:, :187].values
y_test = test_df.iloc[:, 187].values
y_test_binary = test_df['binary_label'].values

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_train_binary = torch.tensor(y_train_binary, dtype=torch.long)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)
y_test_binary = torch.tensor(y_test_binary, dtype=torch.long)

# Create DataLoaders with both labels
batch_size = 1024

train_dataset = TensorDataset(X_train, y_train_binary, y_train)
test_dataset = TensorDataset(X_test, y_test_binary, y_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Hyperparameters
lr = 0.001
num_epochs = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleModel1D(num_classes=2).to(device)  # Assuming binary classification
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    with tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
        for i, (inputs, binary_labels, original_labels) in enumerate(train_loader):
            inputs, binary_labels = inputs.unsqueeze(1).to(device), binary_labels.to(device)  # Add a channel dimension

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, binary_labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss/(i+1))
            pbar.update(1)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Function to extract features
def extract_features(model, data_loader):
    features = []
    labels_list = []
    model.eval()
    with torch.no_grad():
        for inputs, binary_labels, original_labels in data_loader:
            inputs = inputs.unsqueeze(1).to(device)
            outputs = model(inputs)
            features.append(outputs)
            print(outputs.shape)
            labels_list.append(original_labels)
    features = torch.cat(features)
    labels_list = torch.cat(labels_list)
    return features, labels_list

# Extract features and labels
test_features, test_labels = extract_features(model, test_loader)

# Calculate cosine similarity
distances = cosine_similarity(test_features.detach().cpu(), test_features.detach().cpu())
np.fill_diagonal(distances, float('-inf'))

# Functions for top-k accuracy
def find_max_indices(arr, k):
    if k > len(arr):
        raise ValueError("k cannot be greater than the size of the array")
    idx = np.argpartition(arr, -k)[-k:]
    sorted_idx = np.argsort(arr[idx])[::-1]
    return idx[sorted_idx]

def compute_top_k_accuracy(distances, all_ids, k):
    correct = 0
    for id_, elem in enumerate(distances, start=0):
        indices = find_max_indices(elem, k)
        candidates = [all_ids[indices[i]].item() for i in range(len(indices))]
        if all_ids[id_].item() in candidates:
            correct += 1
    accuracy = correct / len(distances)
    return accuracy

# Compute top-k accuracy
real_top_accuracies = []
for k in range(1, 5):
    accuracy = compute_top_k_accuracy(distances, test_labels, k)
    real_top_accuracies.append(accuracy)
    print(f"Top-{k} : {accuracy:.4f}")


Epoch 1/10: 100%|██████████| 86/86 [00:02<00:00, 30.23batch/s, loss=0.489]


Epoch 1/10, Loss: 0.48896429219911264


Epoch 2/10: 100%|██████████| 86/86 [00:03<00:00, 21.94batch/s, loss=0.382]


Epoch 2/10, Loss: 0.3816742508910423


Epoch 3/10: 100%|██████████| 86/86 [00:03<00:00, 28.43batch/s, loss=0.298]


...

Epoch 8/10, Loss: 0.18042369377474451


Epoch 9/10: 100%|██████████| 86/86 [00:02<00:00, 31.25batch/s, loss=0.17]


Epoch 9/10, Loss: 0.17018144941607186


Epoch 10/10: 100%|██████████| 86/86 [00:03<00:00, 28.62batch/s, loss=0.165]


Epoch 10/10, Loss: 0.16482801354208657
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([1024, 2])
torch.Size([388, 2])
Top-1 : 0.6826
Top-2 : 0.9101
Top-3 : 0.9315
Top-4 : 0.9440
